## Deep learning

![](https://www.frontiersin.org/files/Articles/420104/fgene-10-00214-HTML/image_m/fgene-10-00214-g001.jpg)

The network structure of a deep learning model. Here we select a network structure with two hidden layers as an illustration, where X nodes constitute the input layer, Hs for the hidden layers, Y for the output layer, and f (·) denotes an activation function.

[Interactive example](https://cs.stanford.edu/people/karpathy/convnetjs/demo/classify2d.html)

![](https://www.frontiersin.org/files/Articles/420104/fgene-10-00214-HTML/image_m/fgene-10-00214-g002.jpg)

The general analysis procedure commonly adopted in deep learning, which covers training data preparation, model construction, hyperparameter fine-tuning (in training loop), prediction and performance evaluation. Basically, it still follows the requisite schema in machine learning.

## Geometric Deep Learning Models

Typically, graph neural networks work by passing messages between nodes in the graph whereever there is an edge, and then aggregating these messages at each node.

![](https://www.aritrasen.com/wp-content/uploads/2022/11/msg_1.jpg)

This is done iteratively for a number of layers, and then the final node representations are used to make predictions. This way, the model can learn to take into account the structure of the graph when making predictions and information flows between connected nodes.



![](https://ars.els-cdn.com/content/image/1-s2.0-S2666651021000012-gr2_lrg.jpg)

Sometimes you may even change the graph structure itself, by pooling nodes or edges, or by adding new edges or nodes. This can be useful for tasks like graph classification, where you want to make a prediction about the entire graph, or for tasks like graph generation, where you want to create new graphs that are similar to the ones you've seen before.

![](https://www.researchgate.net/profile/Lavender-Jiang-2/publication/343441194/figure/fig4/AS:921001206509568@1596595207558/Graph-pooling-and-graph-aggregation-Graph-pooling-left-accepts-a-graph-signal-and.ppm)

So, depending on the way messages are passed, aggregated, pooled, and transformed, there are a number of different architectures that can be used to build graph neural networks. Here's a comprehensive [review](https://www.sciencedirect.com/science/article/pii/S2666651021000012) and a [book](https://arxiv.org/pdf/2104.13478.pdf) to learn more about the different architectures, how they work, and how they can be used for different tasks.

![](https://ars.els-cdn.com/content/image/1-s2.0-S2666651021000012-gr3_lrg.jpg)

In [2]:
from torch_geometric import nn as graph_nn
from torch import nn
from src import dataloader
import torch
import lightning

## PyTorch-Geometric models and layers

In this notebook, we'll take a look at some of the different architectures and building block layers that are implemented in the `torch-geometric` library, and how they can be used to build and train models for graph-based tasks.

- [torch_geometric.nn](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html) has a variety of graph layers that can be used to build custom GNN architectures. These include:
    - [Convolutional layers](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#convolutional-layers): These define how the message passing step is accomplished across edges in the graph. `GCNConv` is a simple example of a graph convolution layer, while `GATConv` is a more complex example with attention mechanisms.
    - [Aggregation Operators](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#aggregation-operators): These define how messages are aggregated at each node. 
    - [Pooling layers](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#id45): These define how nodes are aggregated into a single node.

- [torch_geometric.nn.models](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#models) has more complex model architectures with a variety of of these layers already defined and combined inside.
- The [PyGModelHubMixin](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.model_hub.PyGModelHubMixin) class can be used to load pre-trained models or other model architectures from the [HuggingFace Model Hub](https://huggingface.co/models?pipeline_tag=graph-ml&sort=trending)

We can load our datamodule from the previous notebook and get a train batch to test out these layers and models:

In [3]:
datamodule = dataloader.ProteinGraphDataModule("./test_data", "dataset.txt")
datamodule.prepare_data()
datamodule.setup("fit")

train_loader = datamodule.train_dataloader()
example_train_batch = next(iter(train_loader))

Processing...
Done!
Processing...
Done!


## Graph neural network layers

Here's how we would define a graph convolutional layer that takes amino acid one hot embeddings as input node features along with the edge index to define the graph, and converts them to a 64-dimensional embedding via convolution operations across the graph:

In [4]:
layer = graph_nn.GCNConv(in_channels=20, out_channels=64)
example_output = layer(example_train_batch.amino_acid_one_hot.float(), example_train_batch.edge_index)
example_output.shape

torch.Size([1181, 64])

**Try out some of the other layers in the torch_geometric.nn module!**

## Graph neural network models

We can also try out some of the pre-defined models in the `torch_geometric.nn.models` module, such as the `GAT` model which applies a series of `GATv2Conv` layers that uses attention mechanisms to weight the importance of different nodes in the graph when aggregating information from neighbors, followed by a Linear layer to convert the node embeddings to a 64-dimensional output.

In [5]:
model = graph_nn.GAT(in_channels=20,
                     hidden_channels=32,
                     num_layers=3,
                     heads=2,
                     out_channels=64,
                     dropout=0.01,
                     jk="last", 
                     v2=True)
print(graph_nn.summary(model, example_train_batch.amino_acid_one_hot.float(), example_train_batch.edge_index))
print(model(example_train_batch.amino_acid_one_hot.float(), example_train_batch.edge_index).shape)

+---------------------+------------------------+----------------+----------+
| Layer               | Input Shape            | Output Shape   | #Param   |
|---------------------+------------------------+----------------+----------|
| GAT                 | [1181, 20], [2, 10320] | [1181, 64]     | 7,872    |
| ├─(dropout)Dropout  | [1181, 32]             | [1181, 32]     | --       |
| ├─(act)ReLU         | [1181, 32]             | [1181, 32]     | --       |
| ├─(convs)ModuleList | --                     | --             | 5,760    |
| │    └─(0)GATv2Conv | [1181, 20], [2, 10320] | [1181, 32]     | 1,408    |
| │    └─(1)GATv2Conv | [1181, 32], [2, 10320] | [1181, 32]     | 2,176    |
| │    └─(2)GATv2Conv | [1181, 32], [2, 10320] | [1181, 32]     | 2,176    |
| ├─(norms)ModuleList | --                     | --             | --       |
| │    └─(0)Identity  | [1181, 32]             | [1181, 32]     | --       |
| │    └─(1)Identity  | [1181, 32]             | [1181, 32]     | --       |

**Try out some of the other models in the torch_geometric.nn module!**

We can combine layers into custom architectures. Here is an example of a simple architecture that uses a GATConv layer and a GCNConv layer with some activation functions in between, and finally a linear layer to convert the 64-dimensional node embeddings to one value per node.

In [6]:
model = graph_nn.Sequential('x, edge_index', [
    (graph_nn.GATConv(in_channels=20, out_channels=64, heads=2, concat=False), 'x, edge_index -> x'),
    nn.ReLU(inplace=True),
    (graph_nn.GCNConv(in_channels=64, out_channels=64), 'x, edge_index -> x'),
    nn.ReLU(inplace=True),
    nn.Linear(64, 1),
])

print(graph_nn.summary(model, example_train_batch.amino_acid_one_hot.float(), example_train_batch.edge_index))

+---------------------+------------------------+----------------+----------+
| Layer               | Input Shape            | Output Shape   | #Param   |
|---------------------+------------------------+----------------+----------|
| Sequential          | [1181, 20], [2, 10320] | [1181, 1]      | 7,105    |
| ├─(module_0)GATConv | [1181, 20], [2, 10320] | [1181, 64]     | 2,880    |
| ├─(module_1)ReLU    | [1181, 64]             | [1181, 64]     | --       |
| ├─(module_2)GCNConv | [1181, 64], [2, 10320] | [1181, 64]     | 4,160    |
| ├─(module_3)ReLU    | [1181, 64]             | [1181, 64]     | --       |
| ├─(module_4)Linear  | [1181, 64]             | [1181, 1]      | 65       |
+---------------------+------------------------+----------------+----------+


## Defining losses

In order to train such models with our data for our task of interface residue prediction, we need to define a loss function that takes the output of the model (the prediction) and the target labels and computes a loss value that the optimizer can use to update the model parameters. A typical choice for binary classification tasks is the binary cross entropy loss, which is implemented in PyTorch as `torch.nn.BCEWithLogitsLoss`. This loss function takes the raw output of the model and the target labels, and applies the sigmoid function to the model output to get the predicted probabilities, and then computes the binary cross entropy loss between the predicted probabilities and the target labels, defined as

$$
\text{loss} = -\frac{1}{N} \sum_{i=1}^N \left[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right]
$$

where $N$ is the number of residues, $y_i$ is the target label for residue $i$ (1 if it's an interface residue, 0 if not), and $p_i$ is the predicted probability for residue $i$.

In [17]:
nn.BCEWithLogitsLoss()(model(example_train_batch.amino_acid_one_hot.float(), example_train_batch.edge_index), 
                       example_train_batch.interface_label.view(-1, 1).float())

tensor(0.6860, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

## Training a model

Given a model that predicts interface probabilities and a loss that compared them with the true interface labels, what we now need is a training loop that will iterate over the training data in batches, compute the loss, and use an optimizer to update the model parameters based on the loss value.

All of this is encapsulated within the `LightningModule` class in PyTorch Lightning.

![](https://lightningaidev.wpengine.com/wp-content/uploads/2023/10/pl-walk-lit-module.png)

In [23]:
class InterfaceModule(lightning.LightningModule):
    """
    LightningModule wrapping a GAT model.
    """
    def __init__(self, in_channels=20, hidden_channels=32, num_layers=2, heads=2, out_channels=1, dropout=0.01, jk="last"):
        super().__init__()
        self.model = graph_nn.GAT(in_channels=in_channels,
                         hidden_channels=hidden_channels,
                         num_layers=num_layers,
                         heads=heads,
                         out_channels=out_channels,
                         dropout=dropout,
                         jk=jk, v2=True)
        self.loss_function = nn.BCEWithLogitsLoss()

    def forward(self, node_attributes, edge_index):
        return self.model(node_attributes, edge_index)

    def training_step(self, batch, batch_idx):
        out = self(batch.amino_acid_one_hot.float(), batch.edge_index)
        loss = self.loss_function(out, batch.interface_label.float().view(-1, 1))
        self.log('train_loss', loss, on_step=True, on_epoch=True, sync_dist=True,
                 batch_size=batch.batch_size)
        return loss

    def configure_optimizers(self):
      return torch.optim.Adam(params=self.model.parameters(), lr=0.001, weight_decay=0.0001)

The `Trainer` class then combines the training loop defined in the LightningModule with the data loading functions in the LightningDataModule. We set the `max_epochs` to 5, meaning that the training loop will iterate over the entire training data 5 times, updating the model parameters with each batch.

In [24]:
model = InterfaceModule()
datamodule = dataloader.ProteinGraphDataModule("./test_data", "dataset.txt")
trainer.fit(model=model, datamodule=datamodule)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Processing...
Done!
Processing...
Done!

  | Name          | Type              | Params
----------------------------------------------------
0 | model         | GAT               | 3.6 K 
1 | loss_function | BCEWithLogitsLoss | 0     
----------------------------------------------------
3.6 K     Trainable params
0         Non-trainable params
3.6 K     Total params
0.014     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


**Congratulations, your model is training!**

The next step is to monitor the performance of the model on the validation data (and maybe even stop training when the performance stops improving). We'd probably like to see some metrics beyond the loss value (like accuracy, precision, recall) and how those change over time, both on the training data and the validation data to make sure the model is learning something useful and not overfitting. All of this needs more complex logging and monitoring than what we've done so far, covered in the next notebook.

### [PeSTo: parameter-free geometric deep learning for accurate prediction of protein binding interfaces](https://www.nature.com/articles/s41467-023-37701-8)

![PeSTO](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41467-023-37701-8/MediaObjects/41467_2023_37701_Fig1_HTML.png)

a Primary geometric transformer acting on the scalar and vectorial state of an atom at layer t. The interactions between the central atom and the nearest neighbors are encoded. A transformer is used to decode and filter the interactions information and to compute the new state of the central atom (Supplementary Algorithm 1). b The architecture of PeSTo for the prediction of interaction interfaces. The model is composed of multiple layers of geometric transformers with a set number of nearest neighbors (nn) and residual connections. The structure is reduced to a residue representation through an attention-based geometric pooling (Supplementary Algorithm 2). The residue states are collapsed, and the final prediction is computed from a multi-layer perceptron (MLP). c Example of application of the primary geometric transformer to all atoms in a structure.

---

### [Structure-based protein function prediction using graph convolutional networks (DeepFri)](https://www.nature.com/articles/s41467-021-23303-9)
![DeepFri](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41467-021-23303-9/MediaObjects/41467_2021_23303_Fig1_HTML.png?as=webp)
a LSTM language model, pre-trained on ~10 million Pfam protein sequences, used for extracting residue-level features of PDB sequence. b Our GCN with three graph convolutional layers for learning complex structure–function relationships.

---

### [Deciphering interaction fingerprints from protein molecular surfaces using geometric deep learning](https://www.nature.com/articles/s41592-019-0666-6)
![MaSIF](https://media.springernature.com/full/springer-static/esm/art%3A10.1038%2Fs41592-019-0666-6/MediaObjects/41592_2019_666_Fig15_ESM.jpg?as=webp)
Patches are fed through convolutional layers followed by a series of fully connected layers (FC5, FC4, FC2), and finally a sigmoid cross-entropy loss.

---

### [ScanNet: an interpretable geometric deep learning model for structure-based protein binding site prediction](https://www.nature.com/articles/s41592-022-01490-7)
![ScanNet](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41592-022-01490-7/MediaObjects/41592_2022_1490_Fig1_HTML.png)
ScanNet inputs are the primary sequence, tertiary structure and, optionally, position–weight matrix computed from a MSA of evolutionarily related proteins. First, for each atom, neighboring atoms are extracted from the structure and positioned in a local coordinate frame (top left). The resulting point cloud is passed through a set of trainable, linear filters detecting specific spatio-chemical arrangements (top middle), yielding an atomic-scale representation (top right). After aggregation of the atomic representation at the amino acid level and concatenation with amino acid attributes, the process is reiterated with amino acids to obtain a representation of an amino acid (bottom). The latter is projected and locally averaged for residue-wise classification.

---